In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
import os

#OpenAI에서 발급받은 API Key를 'sk-...' 부분에 기입합니다.
os.environ["OPENAI_API_KEY"] = 'sk-...'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap = 200)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI의 임베딩 모델 사용
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS DB 생성 후 저장
faiss_store = FAISS.from_documents(docs, embedding)
faiss_store.save_local("/content/DB")

In [ ]:
# 저장된 DB경로 지정 후, DB 로드
persist_directory = "/content/DB"
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [ ]:
# FAISS 리트리버 생성
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

# 관련있는 문서를 수집 후, Chatgpt로 최종 답변까지 수행
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.2, model="gpt-4o"),
    chain_type="stuff",
    retriever=faiss_retriever,
    return_source_documents=True # 답변에 사용된 source document도 보여주도록 설정
)

In [ ]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량이 어느정도야?")